In [6]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import os
import seaborn as sns
import random 
import scipy as sc
import statsmodels.api as sm
import scipy.stats as stats
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy as cp
import pygrib
import cfgrib
import glob
import sys
import stormeunice as eun

sns.set_theme(style="white")
sns.set_style("white")

random.seed(10)

# Description

This script creaates a plot to disply the behaviour of the ensemble in terms of wind gusts and mean sea level pressure.

# Import data

In [2]:
lon_min = -15
lon_max = 5
lat_min = 45
lat_max = 60

In [3]:
# ERA5

era5_2022 = xr.open_dataset('/gf3/predict2/AWH012_LEACH_NASTORM/DATA/ERA5/EU025/sfc/2022.nc')
era5_GUST = era5_2022.fg10.sel(time=slice('2022-02-07','2022-02-22'),latitude=slice(lat_max,lat_min),longitude=slice(lon_min, lon_max)).resample(time='1d').max().load()

In [4]:
# EPS

directory = '/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/ENS/EU025/sfc/'
eps = []  # making sure to start from a clean slate

for ind, cont in enumerate(['cf', 'pf']):
    for files in glob.glob(directory+cont+'/*.nc'):
        print(files)
        if len(eps) == ind+1:
            data = xr.open_dataset(files)
            data = eun.data.Data.preproc_ds(data.get(['fg10', 'msl']))  # preprocessing just one variable for speed
            eps[ind] = xr.concat([eps[ind], data], dim = 'inidate')
        else:
            eps.append(eun.data.Data.preproc_ds(xr.open_dataset(files).get(['fg10', 'msl'])))

eps = xr.concat(eps, dim = 'number')

/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/ENS/EU025/sfc/cf/1_2022-02-16.nc
/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/ENS/EU025/sfc/cf/1_2022-02-14.nc
/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/ENS/EU025/sfc/cf/1_2022-02-10.nc
/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/ENS/EU025/sfc/pf/1_2022-02-14.nc
/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/ENS/EU025/sfc/pf/1_2022-02-10.nc
/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/ENS/EU025/sfc/pf/1_2022-02-16.nc


# Calculate aggregated fields

In [13]:
ensmean_mslp = eps.sel(time = '2022-02-18 12').mean(dim = 'number').squeeze()
ensfrac_fg10 = eps.sel(time = '2022-02-18 12').count(dim = 'number').squueze()